## Importing required modules

In [1]:
from crtsh import crtshAPI ## search certificate /
import json
import time
import csv
from waybackpy import WaybackMachineCDXServerAPI ## search historical copies of website
import socket
import dns.resolver
import whois # query and response protocol that is often used for querying databases that store registered domain names.
import datetime
import pandas as pd
import os



def download_page(url):

    #TODO: Integrate shot-scraper CLI calls
#     print("current download_page url:",url)
    !shot-scraper {url}
#     print("download page completed")

    return 0

# Returns URL of latest archive page, timestamp of latest archive page
def download_archived_page(url):

    try:
        user_agent = "Mozilla/5.0 (Windows NT 5.1; rv:40.0) Gecko/20100101 Firefox/40.0"
        cdx_api = WaybackMachineCDXServerAPI(url, user_agent)

        newest = cdx_api.newest()

        return [newest.archive_url, newest.timestamp]

    except:
        print("download_archieved_page exception triggered")

        return ['', '']

# Returns Subject CN, SAN, Issuer Details, Issued Date, Earliest Issue Date of the first cert, Number of Certs
def download_cert(domain_name):

    if not os.path.exists("downloaded_certs"):
            os.makedirs("downloaded_certs")
        
    try:
        cert_json = crtshAPI().search(domain_name)
#         print(cert_json)
        
#         print(cert_json[len(cert_json)-1])
        
        
        with open("downloaded_certs/" + domain_name + ".json", "w") as outfile:
            json.dump(cert_json, outfile)

        return [cert_json[0]['common_name'], cert_json[0]['name_value'], cert_json[0]['issuer_name'], cert_json[0]['not_before'], len(cert_json)]

    except:

        print("download_cert exception triggered")
        return ['', '', '', '', '', '']



# Returns Unique CNs, Set of CNs, Unique Issuers, Set of Issuers, Unique AltName Values, Min of AltName Count, Max of AltName Count
def process_downloaded_cert(domain_name):

    try:
        # Opening JSON file
        f = open("downloaded_certs/" + domain_name + ".json")
        data = json.load(f)

        SubjectCN_set = set()
        Issuer_set = set()
        SerialNo_set = set()
        AltName_set = set()
        AltName_count_min = 99999
        AltName_count_max = 0

        if (len(data) == 0):
            return ['', '', '', '', '', '', '']


        for i in data:
            #print(i)

            SubjectCN_set.add(i["common_name"])
            Issuer_set.add(i["issuer_name"])
            SerialNo_set.add(i["serial_number"])
            AltName_set.add(i["name_value"])


            NumOfAltNamesInside = (i["name_value"].count("\n"))+1
            if (NumOfAltNamesInside < AltName_count_min):
                AltName_count_min = NumOfAltNamesInside

            if (NumOfAltNamesInside > AltName_count_max):
                AltName_count_max = NumOfAltNamesInside

        # Closing file
        f.close()

        return [len(SubjectCN_set), SubjectCN_set, len(Issuer_set), Issuer_set, len(AltName_set), AltName_count_min, AltName_count_max]

    except:
        print("process_downloaded_cert exception triggered")
        return ['', '', '', '', '', '', '']

# Returns CNAME, A IPs, Nameservers
def get_dns_info(domain_name):

    try:
        dns_info = socket.gethostbyname_ex(domain_name)
        print(dns_info)
        nameservers = dns.resolver.query(domain_name, 'NS')
        nameserver_list = [i.to_text() for i in nameservers]

        return [dns_info[1], dns_info[2], nameserver_list]

    except:

        print("get_dns_info exception triggered")
        return [[], [], []]


# Returns Registra, Name, Org, Created Date, Updated Date
def get_domain_whois_info(domain_name):

    if not os.path.exists("downloaded_whois"):
            os.makedirs("downloaded_whois")
            
    try:
        w = whois.whois(domain_name)
        with open("downloaded_whois/" + domain_name + ".txt", "w") as outfile:
            outfile.write(str(w.text))

        return [w.registrar, w.name, w.org, w.creation_date, w.updated_date]

    except:

        print("get_domain_whois_info() exception triggered")
        return ['', '', '', '', '']



with open('to_process.csv', newline='') as inputfile:
    with open('output_certVariations.csv', 'w', newline='') as outputfile:
        domain_list = csv.reader(inputfile, delimiter=',')
        output_writer = csv.writer(outputfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)

        counter = 0

        for row in domain_list:
            print("Processing: #" + str(counter) + " - " + row[0])
#             print("Printing Row: ", row)

            cert_list = download_cert(row[0])
            archivedpage_list = download_archived_page(row[0])
            dns_list = get_dns_info(row[0])
            whois_list = get_domain_whois_info(row[0])
            image = download_page(row[0])

            cert_variations = process_downloaded_cert(row[0])

#             output_writer.writerow(row + [datetime.datetime.now()] + cert_list + archivedpage_list + dns_list + whois_list)

            output_writer.writerow(row + [datetime.datetime.now()] + cert_variations)

            outputfile.flush()

            counter += 1
            #time.sleep(4)



print("completed")
exit(0)




ModuleNotFoundError: No module named 'crtsh'

## VT Call

In [6]:
API_KEY = '0d9fdb6e32d74b9d12e3d894309531838c3aabe8d66b049fd3a7976fbedf2c68'  #@param  {type: "string"}

r = requests.get("https://www.virustotal.com/api/v3/domains/www.amazon.com", headers={"x-apikey":API_KEY})



In [14]:
# from crtsh import crtshAPI ## search certificate /
import json
import time
import csv
# from waybackpy import WaybackMachineCDXServerAPI ## search historical copies of website
import socket
import dns.resolver
# import whois # query and response protocol that is often used for querying databases that store registered domain names.
import datetime
import pandas as pd
import os

def get_dns_info(domain_name):

    try:
        dns_info = socket.gethostbyname_ex(domain_name)
        print("dns_info:", dns_info)
        nameservers = dns.resolver.query(domain_name, 'NS')
        print("nameservers:", nameservers.response)
        print(type(nameservers))

        nameserver_list = [i.to_text() for i in nameservers]

        return [dns_info[1], dns_info[2], nameserver_list]

    except Exception as e:
        print(e)
        print("get_dns_info exception triggered")
        return [[], [], []]
    
    
get_dns_info("www.amazon.com")

dns_info: ('e15316.dsca.akamaiedge.net', ['www.amazon.com', 'tp.47cf2c8c9-frontier.amazon.com', 'www.amazon.com.edgekey.net'], ['119.56.1.86'])
nameservers: id 48620
opcode QUERY
rcode NOERROR
flags QR RD RA
;QUESTION
www.amazon.com. IN NS
;ANSWER
www.amazon.com. 276 IN CNAME tp.47cf2c8c9-frontier.amazon.com.
tp.47cf2c8c9-frontier.amazon.com. 58 IN CNAME d3ag4hukkh62yn.cloudfront.net.
d3ag4hukkh62yn.cloudfront.net. 1485 IN NS ns-2021.awsdns-60.co.uk.
d3ag4hukkh62yn.cloudfront.net. 1485 IN NS ns-130.awsdns-16.com.
d3ag4hukkh62yn.cloudfront.net. 1485 IN NS ns-1144.awsdns-15.org.
d3ag4hukkh62yn.cloudfront.net. 1485 IN NS ns-824.awsdns-39.net.
;AUTHORITY
;ADDITIONAL
<class 'dns.resolver.Answer'>


C:\Users\Jun Rong\AppData\Local\Temp\ipykernel_21476\4224909885.py:18: DeprecationWarning: please use dns.resolver.resolve() instead
  nameservers = dns.resolver.query(domain_name, 'NS')


[['www.amazon.com',
  'tp.47cf2c8c9-frontier.amazon.com',
  'www.amazon.com.edgekey.net'],
 ['119.56.1.86'],
 ['ns-2021.awsdns-60.co.uk.',
  'ns-130.awsdns-16.com.',
  'ns-1144.awsdns-15.org.',
  'ns-824.awsdns-39.net.']]

In [5]:
from crtsh import crtshAPI ## search certificate /
import json
import time
import csv
from waybackpy import WaybackMachineCDXServerAPI ## search historical copies of website
import socket
import dns.resolver
import whois # query and response protocol that is often used for querying databases that store registered domain names.
import datetime
import pandas as pd
import os
import requests


API_KEY = '0d9fdb6e32d74b9d12e3d894309531838c3aabe8d66b049fd3a7976fbedf2c68'  #@param  {type: "string"}


## NOTE: ONLY STARTS FROM ROW 1 AND NOT 0 PLEASE EDIT

def process_domainlist(filename_to_process, columnIndex):
    
    with open(filename_to_process + ".csv", newline='') as inputfile:

        if not os.path.exists("downloaded_vtresponse"):
                os.makedirs("downloaded_vtresponse")

        with open(filename_to_process + "_tracker.csv", 'w', newline='') as outputfile:
            ip_list = csv.reader(inputfile, delimiter=',')
            output_writer = csv.writer(outputfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)

            counter = 0

            for row in ip_list:

                if counter == 0:
                    output_writer.writerow(row + ["Time Run"])
                    outputfile.flush()
                    counter += 1
                    continue

                print("Processing: #" + str(counter) + " - " + row[columnIndex])

                #Get an IP address Report
#                 r = requests.get("https://www.virustotal.com/api/v3/ip_addresses/"+row[0], headers={"x-apikey":API_KEY})
                r = requests.get("https://www.virustotal.com/api/v3/domains/" + row[0], headers={"x-apikey":API_KEY})


                # TODO: Make generalised and incorporate timestamp in foldername
                now = datetime.datetime.now()
                dt_string = now.strftime("%d%m%Y")
                
                if not os.path.exists("downloaded_vtresponse/" + dt_string):
                    os.makedirs("downloaded_vtresponse/" + dt_string)
#                 with open("downloaded_vtresponse/" + row[columnIndex] +".json", "w") as outfile:
                with open("downloaded_vtresponse/" + dt_string + "/" + row[columnIndex] +".json", "w") as outfile:

                    outfile.write(r.text)

                output_writer.writerow(row + [datetime.datetime.now()])

                outputfile.flush()

                counter += 1
                time.sleep(16)
                
process_domainlist("to_process", 0)


Processing: #1 - www.facebook.com
Processing: #2 - www.amazon.com


In [12]:
get_dns_info("www.amazon.com")

get_dns_info for: www.amazon.com


C:\Users\puddi\AppData\Local\Temp\ipykernel_20044\1322193862.py:116: DeprecationWarning: please use dns.resolver.resolve() instead
  nameservers = dns.resolver.query(domain_name, 'NS')


[['www.amazon.com', 'tp.47cf2c8c9-frontier.amazon.com'],
 ['13.33.37.125'],
 ['ns-130.awsdns-16.com.',
  'ns-2021.awsdns-60.co.uk.',
  'ns-824.awsdns-39.net.',
  'ns-1144.awsdns-15.org.']]

In [13]:
get_dns_info("apple.com")

get_dns_info for: apple.com


C:\Users\puddi\AppData\Local\Temp\ipykernel_20044\1322193862.py:116: DeprecationWarning: please use dns.resolver.resolve() instead
  nameservers = dns.resolver.query(domain_name, 'NS')


[[],
 ['17.253.144.10'],
 ['d.ns.apple.com.', 'c.ns.apple.com.', 'a.ns.apple.com.', 'b.ns.apple.com.']]

In [18]:
get_domain_whois_info("www.apple.com")

{
  "domain_name": [
    "APPLE.COM",
    "apple.com"
  ],
  "registrar": "CSC CORPORATE DOMAINS, INC.",
  "whois_server": "whois.corporatedomains.com",
  "referral_url": null,
  "updated_date": [
    "2023-02-16 06:14:38",
    "2023-02-16 01:14:38"
  ],
  "creation_date": [
    "1987-02-19 05:00:00",
    "1987-02-19 00:00:00"
  ],
  "expiration_date": "2024-02-20 05:00:00",
  "name_servers": [
    "A.NS.APPLE.COM",
    "B.NS.APPLE.COM",
    "C.NS.APPLE.COM",
    "D.NS.APPLE.COM",
    "a.ns.apple.com",
    "b.ns.apple.com",
    "c.ns.apple.com",
    "d.ns.apple.com"
  ],
  "status": [
    "clientTransferProhibited https://icann.org/epp#clientTransferProhibited",
    "serverDeleteProhibited https://icann.org/epp#serverDeleteProhibited",
    "serverTransferProhibited https://icann.org/epp#serverTransferProhibited",
    "serverUpdateProhibited https://icann.org/epp#serverUpdateProhibited",
    "clientTransferProhibited http://www.icann.org/epp#clientTransferProhibited",
    "serverDeleteP

['CSC CORPORATE DOMAINS, INC.',
 'Domain Administrator',
 'Apple Inc.',
 [datetime.datetime(1987, 2, 19, 5, 0), datetime.datetime(1987, 2, 19, 0, 0)],
 [datetime.datetime(2023, 2, 16, 6, 14, 38),
  datetime.datetime(2023, 2, 16, 1, 14, 38)]]

In [14]:
get_domain_whois_info("13.33.37.125")

get_domain_whois_info() exception triggered
No match for "R.CLOUDFRONT.NET".
>>> Last update of whois database: 2023-02-20T06:18:34Z <<<

NOTICE: The expiration date displayed in this record is the date the
registrar's sponsorship of the domain name registration in the registry is
currently set to expire. This date does not necessarily reflect the expiration
date of the domain name registrant's agreement with the sponsoring
registrar.  Users may consult the sponsoring registrar's Whois database to
view the registrar's reported date of expiration for this registration.

TERMS OF USE: You are not authorized to access or query our Whois
database through the use of electronic processes that are high-volume and
automated except as reasonably necessary to register domain names or
modify existing registrations; the Data in VeriSign Global Registry
Services' ("VeriSign") Whois database is provided by VeriSign for
information purposes only, and to assist persons in obtaining information
about o

['', '', '', '', '']